In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [10]:
class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None, verbose = False):
        
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 1e-10
                        num_iter: Number of Iterations to update coefficient with training data
                        early_stop: Constant control early_stop.
                        intercept: Bool, If we are going to fit a intercept, default True.
                        init_weight: Matrix (n x 1), input init_weight for testing.
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.model_name = 'Linear Regression'

        # store the variables in object scope
        self.alpha: float = alpha
        self.num_iter: int = num_iter
        self.early_stop: float = early_stop
        self.intercept: bool = intercept
        self.init_weight = init_weight  ### For testing correctness.
        
        self.verbose = verbose
        
    
    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and perform gradient descent.
            
            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                Y_train: Matrix or 2-D array. Input target value.
                
                
                TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
                      3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
                      4. Call the gradient_descent function to train.
        """
        
        self.X = np.array(X_train)
        self.y = np.array(y_train).reshape(-1, 1) 
        
        if self.intercept:
            ones = np.ones((self.X.shape[0], 1))
            self.X = np.hstack([ones, self.X])
            self.log(self.X.shape)
            self.log(self.X)
        
        if self.init_weight is None:
            np.random.seed(16)
            self.coef = np.random.uniform(-1, 1, (self.X.shape[1], 1),)
            self.log(self.coef)
        else:
            self.coef = np.array(self.init_weight).reshape(self.X.shape[1], 1)
        
        self.gradient_descent()
        
        # self.coef = self.init_weight #### Please change this after you get the example right.
        
    # unused function. functionality merged into gradient_descent
    # with a manual backward pass, intermediate forward pass calclations are out of scope
    def gradient(self):
        """
            Helper function to calculate the gradient respect to coefficient.
            
            TODO: 5. Think about the matrix format of the gradient of the loss function.
        """
        
        d_loss = 1
        d_diff = d_loss * 2 * (self.y - self.X @ self.coef)
        d_pred = d_diff * -1
        d_coef = (np.asarray(d_pred) * np.asarray(self.X)).sum(axis=0)
        
        # doesn't make sense to have to calculate these values twice
        # i can just keep the gradient inside the gradient_descent function
        grad_coef = (-2 * (self.y - self.X @ self.coef) * self.X).sum(axis=0)
        
        print(np.isclose(d_coef, grad_coef))
        
        self.grad_coef = grad_coef
        
    def gradient_descent(self):
        
        """
            Training function
            
            TODO: 6. Calculate the loss with current coefficients.
                  7. Update the temp_coef with learning rate and gradient.
                  8. Calculate the loss with temp_coef.
                  9. Implement the self adeptive learning rate. 
                      a. If current error is less than previous error, increase learning rate by a factor 1.3. 
                         And update coef, with temp_coef.
                      b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
                         Don't update coef.
                  10. Add the loss to loss list we create.
        """
        
        self.loss = []
        
        for i in range(self.num_iter):
            
            # forward pass
            preds = self.X @ self.coef
            errors = self.y - preds
            squared_errors = (errors**2)
            loss = squared_errors.sum()
            
            self.loss.append(loss)
            
            # backward pass
            dloss = 1
            dsquared_errors = dloss * np.ones_like(squared_errors)
            derrors = dsquared_errors * 2 * errors
            dpreds = derrors * -1
            # dcoef = (dpreds * self.X).sum(axis=0) # for a given coefficients affects a column of x's, which each affect a different prediction
            dcoef = (dpreds * self.X).sum(axis=0).reshape(-1, 1) # for a given coefficients affects a column of x's, which each affect a different prediction

            temp_coef = self.coef - self.alpha * dcoef
            temp_loss = np.sum((self.y - self.X @ temp_coef)**2)          
            pre_error = loss
            
            current_error = temp_loss
            
            ### This is the early stop, don't modify fllowing three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                print(f"EARLY STOP @ {i}")
                self.coef = temp_coef
                return self
            
            if current_error <= pre_error:
                self.alpha *= 1.3
                self.coef = temp_coef
            else:
                self.alpha *= 0.9 # don't update coefs
                
            self.loss.append(loss)
            
            if i % 10000 == 0:
                print('Iteration: ' +  str(i))
                print('Coef: '+ str(self.coef))
                print('Loss: ' + str(current_error))            
                
        return self
    
    def ind_predict(self, x: list):
        """
            Predict the value based on its feature vector x.

            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                result: prediction of given data point
        """
        
        """
            TODO: 11. Implement the prediction function
        """
        # assumes x has intercept added if used here
        return np.array(x) @ self.coef
    
    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Parameter:
            X: Matrix, array or list. Input feature point.
            
            Return:
                ret: prediction of given data matrix
        """
        
        """
        TODO: 12. Make sure add the 1's column like we did to add intercept.
                  13. Revise the following for-loop to call ind_predict to get predictions.

        """
        
        if self.intercept:
            ones = np.ones((X.shape[0], 1))
            X = np.hstack([ones, X])
        
        return X @ self.coef
        
    def log(self, str):
        if self.verbose == True:
            print(str)

In [39]:
def min_max_normalize(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [40]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() +  20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [41]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

Iteration: 0
Coef: [[15]
 [25]]
Loss: 2.931949540902693e+25
Iteration: 10000
Coef: [[15.02874056]
 [30.00748702]]
Loss: 1241.2855503711044
Iteration: 20000
Coef: [[15.05027193]
 [30.00742094]]
Loss: 1230.5235188302652
Iteration: 30000
Coef: [[15.07171179]
 [30.00742077]]
Loss: 1219.8884700011508
Iteration: 40000
Coef: [[15.09305772]
 [30.00735475]]
Loss: 1209.335432801025
Iteration: 50000
Coef: [[15.11431857]
 [30.00732366]]
Loss: 1198.8753634607592
Iteration: 60000
Coef: [[15.13548116]
 [30.00732717]]
Loss: 1188.5593446511898
Iteration: 70000
Coef: [[15.15655031]
 [30.00726026]]
Loss: 1178.2541315732394
Iteration: 80000
Coef: [[15.17752986]
 [30.00726407]]
Loss: 1168.0756711622355
Iteration: 90000
Coef: [[15.19841689]
 [30.00719774]]
Loss: 1157.9580560811191
Iteration: 100000
Coef: [[15.2192214 ]
 [30.00716703]]
Loss: 1147.9435585362207


KeyboardInterrupt: 

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [42]:
np.array(clf.predict(X))

array([[   45.23173789],
       [  195.26753557],
       [  345.30333325],
       [  495.33913093],
       [  645.37492861],
       [  795.41072629],
       [  945.44652397],
       [ 1095.48232165],
       [ 1245.51811933],
       [ 1395.55391701],
       [ 1545.58971468],
       [ 1695.62551236],
       [ 1845.66131004],
       [ 1995.69710772],
       [ 2145.7329054 ],
       [ 2295.76870308],
       [ 2445.80450076],
       [ 2595.84029844],
       [ 2745.87609612],
       [ 2895.9118938 ],
       [ 3045.94769147],
       [ 3195.98348915],
       [ 3346.01928683],
       [ 3496.05508451],
       [ 3646.09088219],
       [ 3796.12667987],
       [ 3946.16247755],
       [ 4096.19827523],
       [ 4246.23407291],
       [ 4396.26987059],
       [ 4546.30566826],
       [ 4696.34146594],
       [ 4846.37726362],
       [ 4996.4130613 ],
       [ 5146.44885898],
       [ 5296.48465666],
       [ 5446.52045434],
       [ 5596.55625202],
       [ 5746.5920497 ],
       [ 5896.62784738],


#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

In [43]:
X_norm = min_max_normalize(X)
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X_norm,y)

Iteration: 0
Coef: [[15]
 [25]]
Loss: 1.2939625376335508e+16
EARLY STOP @ 1308


In [44]:
clf.coef

array([[   50.],
       [29850.]])

In [45]:
X.min(), X.max(), X.max() - X.min()

(1, 996, 995)

In [ ]:
# 29850 / 995 = 30 
# slope checks out

# x - min. every single point is shifted over by the minimum value. line is shifted over by the slope. 

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [46]:
from sklearn.linear_model import LinearRegression

In [54]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X2 = wine[['density','alcohol']]
X2_norm = min_max_normalize(X2)
y2 = wine.quality

In [55]:
lr = LinearRegression()
lr.fit(X2,y2)
## Squared Error with sklearn.
sum((lr.predict(X2) - y2)**2)
lr.intercept_, lr.coef_

(-33.15237986168804, array([34.82170159,  0.39144139]))

In [56]:
lr2 = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight=None)
lr2.fit(X2, y2)


Iteration: 0
Coef: [[-0.55341784]
 [ 0.04632668]
 [ 0.10140291]]
Loss: 5287551503614576.0
Iteration: 10000
Coef: [[0.22566935]
 [0.83438408]
 [0.43902616]]
Loss: 816.584217720002
Iteration: 20000
Coef: [[0.48625587]
 [1.10121317]
 [0.38829796]]
Loss: 806.863747729821


KeyboardInterrupt: 

In [57]:
lr2.coef

array([[0.52015526],
       [1.13655104],
       [0.38208525]])

In [58]:
lr2 = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight=None)
lr2.fit(X2_norm, y2)

Iteration: 0
Coef: [[-0.55341784]
 [ 0.04632668]
 [ 0.10140291]]
Loss: 1312507632361.2021


Iteration: 10000
Coef: [[2.23217576]
 [0.07111898]
 [5.0193558 ]]
Loss: 805.8689303576189
Iteration: 20000
Coef: [[2.23211202]
 [0.09747445]
 [5.01935535]]
Loss: 805.8683657160285
Iteration: 30000
Coef: [[2.23212022]
 [0.12382839]
 [5.0194034 ]]
Loss: 805.8677993221213
Iteration: 40000
Coef: [[2.23205787]
 [0.15018001]
 [5.01940341]]
Loss: 805.8672322634252
Iteration: 50000
Coef: [[2.23202368]
 [0.17654143]
 [5.0194226 ]]
Loss: 805.8666666819033


KeyboardInterrupt: 

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [ ]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

In [ ]:
sum((clf.predict(X) - y)**2)

In [235]:
## Tests 

test = Linear_Regression(intercept=True, verbose=True, alpha=1e-3)
X2 = np.array([[1], [2], [3]])
y2 = np.array([1, 2, 3])
X2, y2
test.fit(X2, y2)
test.coef

(3, 2)
[[1. 1.]
 [1. 2.]
 [1. 3.]]
[[-0.55341784]
 [ 0.04632668]]
Iteration: 0
Coef: [[-0.53865325]
 [ 0.07967055]]
Loss: 18.677391590518212
EARLY STOP


array([[-3.33083181e-15],
       [ 1.00000000e+00]])